<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/QuordleSolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from lxml import html
import re
import numpy as np

In [ ]:
#wikipedia
# words=[]
# freqs=[]
# root=r'https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/PG/2006/04/'
# for start in range(0,40000,10000):
#     content=requests.get(root+f'{start+1}-{start+10000}').content
#     tree = html.fromstring(content)
#     for i in range(1,11):
#         rows=tree.xpath(rf'//table[{i}]//tr')
#         for row in rows[1:]:
#             words.append(''.join(row.xpath(r'td[2]//text()')))  #Some words like we're are contained in 2 a elements
#             freqs.extend(row.xpath(rf'td[3]/text()'))\
# five_char=[]
# pat=re.compile(r'\w{5} \d+')
# for word,freq in zip(words,freqs):
#     comb=word+' '+freq[:-1]
#     m=re.match(pat,comb)
#     if m:
#         five_char.append(m.group(0))

In [23]:
!wget https://raw.githubusercontent.com/IlyaSemenov/wikipedia-word-frequency/master/results/enwiki-20190320-words-frequency.txt -O words.txt

--2022-06-27 20:37:21--  https://raw.githubusercontent.com/IlyaSemenov/wikipedia-word-frequency/master/results/enwiki-20190320-words-frequency.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27465330 (26M) [text/plain]
Saving to: ‘words.txt’

words.txt           100%[===================>]  26.19M  --.-KB/s    in 0.09s   

2022-06-27 20:37:22 (299 MB/s) - ‘words.txt’ saved [27465330/27465330]



In [24]:
five_char=[]
pat=re.compile(r'\w{5} \d+')
with open('words.txt','r') as f:
    for line in f:
        m=re.match(pat,line)
        if m:
            five_char.append(m.group(0))

In [64]:
with open('b.txt','r') as f:
    b=f.read()
tree=html.fromstring(b)

for board in range(1,5):

    incorrects=tree.xpath(f'//div[@aria-label="Game Board {board}"]//div[contains(@aria-label,"incorrect")]/@aria-label')
    diffs=tree.xpath(f'//div[@aria-label="Game Board {board}"]//div[contains(@aria-label,"different")]/@aria-label')
    corrects=tree.xpath(f'//div[@aria-label="Game Board {board}"]//div[contains(@aria-label," correct") and contains(@aria-label,"letter")]/@aria-label')

    absent_set=set()

    for inc in incorrects:
        absent_set.add(inc[1].lower())


    present_set=set()
    excluded_sets=[set(), set(), set(), set(), set()]
    for diff in diffs:
        diff=diff.lower()
        pos=int(diff[12])-1
        present_set.add(diff[1])
        excluded_sets[pos].add(diff[1])

    present=''.join([f'(?=.*{x})' for x in present_set])
    guess = [f'[^{y}]' if len(y)>0 else '\S' for y in 
             [''.join(x) for x in excluded_sets]]


    correct_set=set()
    for correct in corrects:
        correct=correct.lower()
        correct_set.add(correct[1])
        pos=int(correct[12])-1
        guess[pos]=correct[1]

    #letter in correct position can appear in incorrect list if repeated
    absent_set=absent_set-correct_set
    absent_set=absent_set-present_set
    if len(absent_set)>0:
        absent='(?=[^'+''.join(absent_set)+']{5})'
    else:
        absent=''


    guess='(' + ''.join(guess) +r'\s\d+)'
    reg=present+absent+guess
    print()
    print(f'Board {board}', reg)
    pat=re.compile(reg)
    print(f'Board {board}: ',end ='')
    print(re.findall(pat,'\n'.join(five_char))[:5])


Board 1 (?=.*c)(?=.*s)(?=.*a)(?=.*r)(?=[^eugqhbinmtld]{5})(s[^ar][^r][^c][^s]\s\d+)
Board 1: ['scary 4903', 'scarf 2758', 'scarp 1441', 'sparc 1054', 'socar 307']

Board 2 (?=.*s)(?=.*a)(?=.*e)(?=.*r)(?=[^cugqhbinmtld]{5})(ar[^er][^e][^s]\s\d+)
Board 2: ['arose 29405', 'arase 71', 'arske 3']

Board 3 (?=.*u)(?=.*s)(?=[^ecrgqhbianmld]{5})(st[^u]\St\s\d+)
Board 3: ['stout 8128']

Board 4 (?=.*u)(?=[^cerghbamd]{5})(quilt\s\d+)
Board 4: ['quilt 2308']


In [52]:
pat=re.compile(r'(perch\s\d+)')
print(re.findall(pat,'\n'.join(five_char))[:5])

['perch 5708']


In [ ]:
a=set([1,4,6])
b=set([4])
a-b

{1, 6}

In [ ]:
incorrects

["'I' (letter 3) is incorrect",
 "'U' (letter 5) is incorrect",
 "'W' (letter 1) is incorrect",
 "'I' (letter 3) is incorrect",
 "'C' (letter 4) is incorrect",
 "'H' (letter 5) is incorrect",
 "'T' (letter 2) is incorrect",
 "'I' (letter 3) is incorrect",
 "'L' (letter 4) is incorrect",
 "'L' (letter 5) is incorrect",
 "'G' (letter 1) is incorrect",
 "'O' (letter 2) is incorrect",
 "'I' (letter 3) is incorrect",
 "'N' (letter 4) is incorrect",
 "'G' (letter 5) is incorrect",
 "'P' (letter 1) is incorrect",
 "'R' (letter 2) is incorrect",
 "'I' (letter 3) is incorrect",
 "'V' (letter 4) is incorrect",
 "'Y' (letter 5) is incorrect",
 "'N' (letter 1) is incorrect",
 "'I' (letter 3) is incorrect",
 "'V' (letter 4) is incorrect",
 "'C' (letter 2) is incorrect",
 "'R' (letter 4) is incorrect",
 "'F' (letter 5) is incorrect"]